In [ ]:
%run 'common-functions.ipynb'
PARAM_NAME_BEGINS_WITH_FILTER = ["Ds"]


In [ ]:
@dask.delayed
def get_ssm_params(client, account_id, region):
    paginator = client.get_paginator('describe_parameters')
    page_iterator = paginator.paginate(
        ParameterFilters=[
        {
            "Key": "Name",
            "Option": "BeginsWith",
            "Values": PARAM_NAME_BEGINS_WITH_FILTER
        },
    ])
    data = []
    data = [page["Parameters"] for page in page_iterator]
    df = pd.DataFrame([item for sublist in data for item in sublist])
    df["account_id"] = account_id
    df["region"] = region
    return df


In [ ]:
# fetch accounts from metadata once - so dont need to keep taxing ddb in case of issue
accounts = fetch_accounts_from_metadata()


In [ ]:
%%time
# Fetch all cloudwatch log groups across all spokes that dont have a retentionInDays value set
master_df = pd.DataFrame()
dfs = []
print(f"Accounts to process: {len(accounts)}")
for account in accounts:
    account_id = account.get("account")
    account_region = account.get("region")
    client = assume_return_boto_client("ssm", account_id, account_region)
    dfs.append(get_ssm_params(client, account_id, account_region))

print("Computing")
graph = dask.delayed()(dfs)
results = graph.compute()
master_df = pd.concat(results)

In [ ]:
master_df
master_df.to_csv("./ssm-params-filtered.csv")
